<a href="https://colab.research.google.com/github/Corey-Holton/Group_3_Project/blob/main/ML_Model_Learn_MIDI_data_v1__Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#!pip install music21 mido

In [7]:
# Importing Libraries for working with MIDI files
import music21
from music21 import converter, stream, instrument, note, chord
import mido
import numpy as np
import os

In [8]:
# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [15]:
# Specify the File Path
base_path = '../../vickram_exploration/music_training_database'
midi_files = []

In [16]:
print(base_path)

../../vickram_exploration/music_training_database


In [19]:
# Locating all MIDI files within a specified folder on Google Drive
# Then parse the files with music21

for genre_folder in os.listdir(base_path):
  genre_path = os.path.join(base_path, genre_folder)
  if os.path.isdir(genre_path): # check if it's a directory
    for filename in os.listdir(genre_path):
      if filename.endswith(".mid") or filename.endswith(".midi"):
        filepath = os.path.join(genre_path, filename)

        print(f"Parsing {filepath}")
        try:
            midi_file = converter.parse(filepath)
            midi_files.append(midi_file)  # Add parsed data to the list
            print(f"Successfully parsed: {filepath}")
        except Exception as e:
            print(f"Error parsing {filename}: {e}")


# Now you have a list 'midi_files' containing parsed MIDI data

FileNotFoundError: [Errno 2] No such file or directory: '../../vickram_exploration/music_training_database'

In [ ]:
print(f"Total MIDI files found: {len(midi_files)}")

Total MIDI files found: 25


In [ ]:
# Feature Extraction

def extract_features(midi_file):
    notes = []
    for element in midi_file.flat.notesAndRests:
        if isinstance(element, note.Note):
            notes.append([element.pitch.midi, element.duration.quarterLength])
        elif isinstance(element, chord.Chord):
             # Represent chords with a fixed-size list, e.g., always using 3 pitches
            # If a chord has fewer than 3 pitches, pad it with zeros
            chord_pitches = [note.pitch.midi for note in element.notes]
            chord_pitches = chord_pitches + [0] * (3 - len(chord_pitches))  # Pad with zeros
            notes.append(chord_pitches + [element.duration.quarterLength])  # Add duration
    return notes



# # Extract features from all MIDI files
# features = []
# for midi_file in midi_files:
#   features.append(extract_features(midi_file))
#   # Now you have a list 'features' containing extracted features from all MIDI files


In [ ]:
def extract_features(midi_file):
    notes = []
    for element in midi_file.flat.notesAndRests:
        if isinstance(element, note.Note):
            notes.append([element.pitch.midi, element.duration.quarterLength, 0, 0, 0]) # 5 features: pitch, duration, 3 empty slots for chords
        elif isinstance(element, chord.Chord):
            chord_pitches = [note.pitch.midi for note in element.notes]
            # Limit to 3 pitches, pad if less, truncate if more
            chord_pitches = chord_pitches[:3] + [0] * (3 - len(chord_pitches))
            notes.append(chord_pitches + [element.duration.quarterLength, 0])  # 5 features: 3 pitches, duration, 1 empty slot
    return notes

def create_sequences(features, sequence_length):
    sequences = []
    for feature_set in features:
        for i in range(len(feature_set) - sequence_length):
            sequence = feature_set[i:i + sequence_length]
            # Pad the sequence if it's shorter than the desired length
            if len(sequence) < sequence_length:
                sequence = np.pad(sequence, ((0, sequence_length - len(sequence)), (0, 0)), 'constant')
            sequences.append(sequence)
    return sequences

# Create sequences
sequences = create_sequences(features, sequence_length=100)

# Convert sequences to NumPy array
X = np.array(sequences)



ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (31081, 100) + inhomogeneous part.

**Below this Line the code is out of sync with new code above. **

In [ ]:
# Print number of features
print(f"Number of features sets: {len(features)}")
if features: # Check if features is not empty
  print(f"Number of notes in the first set: {len(features[0])}")
  print(f"Number of notes in the second set: {features[1][0:10]}") # Print first 10 elements
else:
  print("Features list is empty.")

Number of features sets: 0
Features list is empty.


In [ ]:
# Creating Numerical Sequence
def create_sequences(features, sequence_length):
    sequences = []
    for feature_set in features:
        for i in range(len(feature_set) - sequence_length):
            sequence = feature_set[i:i+sequence_length]
            sequences.append(sequence)
    return sequences

# Create sequences
sequences = create_sequences(features, sequence_length=100)

# Convert sequences to NumPy array
X = np.array(sequences)

# Normalize the data (optional)
# ... (normalization code)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
# Reshape data fro LSTM input
X = np.reshape(X, (X.shape[0], X.shape[1],1))

# Create the model
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(X.shape[2]))
model.compile(loss='mean_squared_error', optimizer='adam')



IndexError: tuple index out of range